In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Load training data

Load `train.csv` from Kaggle into a pandas DataFrame.

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Set up X and y

NOTE: Keras requires a `numpy` matrix, it doesn't work with `pandas`.

In [4]:
import numpy as np

In [17]:
y=df.label.values
x=df[df.columns[1:]].values

### Preprocessing

1. When dealing with image data, you need to normalize your `X` by dividing each value by the max number of pixels (255).
2. Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
ss=StandardScaler()
x=ss.fit_transform(x)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [22]:
from keras import utils
y=utils.to_categorical(y)

### Train/Test Split

We want to create a validation set that the model will never see to approximate how it's going to do with Kaggle's `test.csv`. Use `sklearn`'s `train_test_split` to do this.

In [27]:
X_train, X_test, y_train, y_test=train_test_split(x,y)

### Create your neural network

Create a neural network using the `Dense` and `Dropout` layers from `keras`. Your activation function for the final output layer needs to be `softmax` to accomidate the ten different classes.

In [40]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_shape=(784,), activation='relu'))
model.add(Dense(4,input_dim=4,activation='relu'))
model.add(Dropout(.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

### Compile your model

Since this is a multiclass classification problem, your loss function is `categorical_crossentropy`.

In [41]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Fit the model

Use your X_test, y_test from the `train_test_split` step for the `validation_data` parameter.

In [43]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=100)

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 9s 295us/step - loss: 1.4853 - val_loss: 0.8471
Epoch 2/10
31500/31500 [==============================] - 9s 278us/step - loss: 1.4795 - val_loss: 0.8066
Epoch 3/10
31500/31500 [==============================] - 9s 286us/step - loss: 1.4643 - val_loss: 0.7840
Epoch 4/10
31500/31500 [==============================] - 10s 304us/step - loss: 1.4639 - val_loss: 0.7838
Epoch 5/10
31500/31500 [==============================] - 9s 294us/step - loss: 1.4531 - val_loss: 0.8089
Epoch 6/10
31500/31500 [==============================] - 9s 296us/step - loss: 1.4507 - val_loss: 0.7856
Epoch 7/10
31500/31500 [==============================] - 9s 294us/step - loss: 1.4414 - val_loss: 0.8119
Epoch 8/10
31500/31500 [==============================] - 9s 294us/step - loss: 1.4302 - val_loss: 0.7754
Epoch 9/10
31500/31500 [==============================] - 9s 300us/step - loss: 1.4259 - val_loss: 0.

### Load in Kaggle's `test.csv`

Be sure to do the **same** preprocessing you did for your training `X`.

In [45]:
test = pd.read_csv('test.csv')
test = test / 255.

### Create your predictions

Use `predict_classes` to get the actual numerical values (0-9).

In [46]:
pred = model.predict_classes(test.values)

28000/28000 [==============================] - 3s 115us/step


### Prepare your submission

1. Add your predictions to a column called `Label`
2. You'll need to manually create the `ImageId` column, which is just a list of 1..[NUMBER OF TEST SAMPLES]

In [47]:
test['Label'] = pred
test['ImageId'] = range(1,test.shape[0] + 1)

### Create your submission csv

Remember to set `index=False`!

In [48]:
test[['ImageId', 'Label']].to_csv('submission.csv', index=False)